# Bridges

From a national data set to local map...

In [2]:
import pandas as pd
import numpy as np
import folium


bridges = pd.read_csv("NY16.csv")
bridges.head()

/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7,36,39,75,96,98,102,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,PROJ_SUFFIX,NBI_TYPE_OF_IMP,DTL_TYPE_OF_IMP,SPECIAL_CODE,STEP_CODE,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING,STATUS_NO_10YR_RULE,CAT10
0,36,1000040,1,2,1,1,0,87,119,44831,...,NaN,NaN,NaN,NaN,NaN,0,NaN,44.1,0,F
1,36,1000090,1,2,1,1,0,87,119,64309,...,0.0,14.0,17.0,NaN,NaN,2,NaN,72.3,2,F
2,36,1000121,1,2,1,1,0,87,119,59223,...,NaN,NaN,NaN,NaN,NaN,2,NaN,56.6,2,F
3,36,1000122,1,2,1,1,0,87,119,59223,...,NaN,NaN,NaN,NaN,NaN,0,NaN,64.1,0,F
4,36,1000140,1,3,1,32,0,11,1,46536,...,0.0,11.0,11.0,NaN,NaN,0,NaN,96.5,0,G


## Bridge Inventory: https://www.fhwa.dot.gov/bridge/nbi.cfm
## Data Dictionary: https://www.fhwa.dot.gov/bridge/nbi/format.cfm
## Guide: https://www.fhwa.dot.gov/bridge/mtguide.pdf
## Data: https://www.fhwa.dot.gov/bridge/nbi/ascii.cfm

### degrees, minutes, seconds: http://www.augustatech.edu/math/molik/DegreesTransparency.pdf


### How many bridges are in New York City?

In [3]:
bx_fips = 5
bk_fips = 47
manhattan_fips = 61
queens_fips = 81
staten_fips = 85
nyc_county_fips = [
    bx_fips,
    bk_fips,
    manhattan_fips,
    queens_fips,
    staten_fips
]

buro_map = [
    'bronx',
    'brooklyn',
    'manhattan',
    'queens',
    'staten island'
]

nyc_bridges = bridges[bridges['COUNTY_CODE_003'].isin(nyc_county_fips)]
nyc_bridges['buro'] = nyc_bridges.apply(lambda x: buro_map[nyc_county_fips.index(x['COUNTY_CODE_003'])], axis=1)
print(len(nyc_bridges['STRUCTURE_NUMBER_008'].unique()))
len(nyc_bridges)

1442


/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1442

### How many 'structurally deficient' bridges are in NYC?

The worst bridges are generally those with a score below 50 percent, are classified as “structurally deficient” due to at least one defect that requires attention. OR

Status field...

1 = Structurally Deficient;
2 = Functionally Obsolete;
0 = Not Deficient;
N = Not Applicable

In [4]:
deficient_nyc_bridges = nyc_bridges[nyc_bridges['STATUS_NO_10YR_RULE'] == 1]
len(deficient_nyc_bridges)

141

### Rank buros by count of deficient bridges

In [5]:
deficient_by_buro = pd.pivot_table(deficient_nyc_bridges,index=["buro"], values=["STRUCTURE_NUMBER_008"], aggfunc=lambda x: len(x.unique()))
deficient_by_buro.reset_index().sort_values(by=['STRUCTURE_NUMBER_008'])

,buro,STRUCTURE_NUMBER_008
4,staten island,4
1,brooklyn,26
2,manhattan,28
3,queens,36
0,bronx,47


### What is the estimated cost to repair these bridges?

In [6]:
#TOTAL_IMP_COST_096
cost_by_buro = pd.pivot_table(deficient_nyc_bridges,index=["buro"], values=["TOTAL_IMP_COST_096"], aggfunc=np.sum)
cost_by_buro.reset_index().sort_values(by=['TOTAL_IMP_COST_096'])


,buro,TOTAL_IMP_COST_096
4,staten island,29183.0
1,brooklyn,1650353.0
3,queens,2056792.0
2,manhattan,2067870.0
0,bronx,2958936.0


In [7]:
cost_count_by_buro = pd.concat((deficient_by_buro, cost_by_buro), axis=1).reset_index()
cost_count_by_buro['TOTAL_IMP_COST_096'] = cost_count_by_buro['TOTAL_IMP_COST_096'].apply(lambda x: '${:,}'.format(x))
cost_count_by_buro

,buro,STRUCTURE_NUMBER_008,TOTAL_IMP_COST_096
0,bronx,47,"$2,958,936.0"
1,brooklyn,26,"$1,650,353.0"
2,manhattan,28,"$2,067,870.0"
3,queens,36,"$2,056,792.0"
4,staten island,4,"$29,183.0"


### Convert lat/lng


In [8]:
deficient_nyc_bridges['lat_degrees'] = deficient_nyc_bridges['LAT_016'].apply(lambda x: float(str(x/1000000).split('.')[0]))
deficient_nyc_bridges['lat_minutes'] = deficient_nyc_bridges['LAT_016'].apply(lambda x: float(str(x/1000000).split('.')[-1][0:2]))
deficient_nyc_bridges['lat_seconds'] = deficient_nyc_bridges['LAT_016'].apply(lambda x: float(str(x/1000000).split('.')[-1][2:4]))
deficient_nyc_bridges['lat'] = deficient_nyc_bridges.apply(lambda x: x['lat_degrees'] + (x['lat_minutes']/60.0) + (x['lat_seconds']/3600.0), axis=1)

deficient_nyc_bridges['long_degrees'] = deficient_nyc_bridges['LONG_017'].apply(lambda x: float(str(x/1000000).split('.')[0]))
deficient_nyc_bridges['long_minutes'] = deficient_nyc_bridges['LONG_017'].apply(lambda x: float(str(x/1000000).split('.')[-1][0:2]))
deficient_nyc_bridges['long_seconds'] = deficient_nyc_bridges['LONG_017'].apply(lambda x: float(str(x/1000000).split('.')[-1][2:4]))
deficient_nyc_bridges['long'] = deficient_nyc_bridges.apply(lambda x: x['long_degrees'] + (x['long_minutes']/60.0) + (x['long_seconds']/3600.0), axis=1)
deficient_nyc_bridges['long'] = deficient_nyc_bridges['long'] * -1
deficient_nyc_bridges.head()[['lat', 'long', 'LAT_016', 'LONG_017']]

/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,lat,long,LAT_016,LONG_017
4386,40.740556,-73.786667,40442699,73471243
4391,40.743889,-73.771389,40443870,73461764
4397,40.747222,-73.761111,40445070,73454053
4418,40.674167,-73.801389,40402750,73480559
4436,40.762500,-73.839722,40454509,73502363


In [12]:
deficient_nyc_bridges[deficient_nyc_bridges['LAT_016'] == 40435073]

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,CAT10,buro,lat_degrees,lat_minutes,lat_seconds,lat,long_degrees,long_minutes,long_seconds,long


### Maps!

https://rawgit.com/CoulterJones/New-School/master/news-narrative-design/spring-2017/level-two/bridges.html

http://rawgit.com/

In [ ]:
bridge_map = folium.Map(location=[40.7128, -74.0059], tiles='Stamen Toner',
                    zoom_start=13)

for row in deficient_nyc_bridges.iterrows():
    idx = row[0]
    obj = row[1]
    latlng = [obj['lat'], obj['long']]
    rating = obj['SUFFICIENCY_RATING']
    marker = folium.CircleMarker(location=latlng, radius=100,
                    color='red',
                    fill_color='red')
    marker.add_to(bridge_map)
bridge_map.save('bridges.html')

In [ ]:
deficient_nyc_bridges[['lat', 'long']].to_csv("nyc-deficient-bridges-locs.csv")